In [1]:
import numpy as np
from pandas import read_csv
from keras.models import Sequential
import pandas as pd
from keras.layers import Conv1D,MaxPool1D,Dense,Flatten,Dropout,LSTM,GlobalAveragePooling1D, Reshape,BatchNormalization,Activation
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.utils import plot_model
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
# set seed to reproduce similar results
np.random.seed(0)

In [2]:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
dataset_file_id = '1xnD6OXIMNOChR4V9DKO-Jjsb7QNMUjc0'

In [5]:
downloaded = drive.CreateFile({'id':dataset_file_id})
downloaded.GetContentFile('UCI HAR Dataset.zip')

In [6]:
!unzip "/content/UCI HAR Dataset.zip" -d "/content/"


Archive:  /content/UCI HAR Dataset.zip
   creating: /content/UCI HAR Dataset/
  inflating: /content/UCI HAR Dataset/.DS_Store  
   creating: /content/__MACOSX/
   creating: /content/__MACOSX/UCI HAR Dataset/
  inflating: /content/__MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: /content/UCI HAR Dataset/activity_labels.txt  
  inflating: /content/__MACOSX/UCI HAR Dataset/._activity_labels.txt  
  inflating: /content/UCI HAR Dataset/features.txt  
  inflating: /content/__MACOSX/UCI HAR Dataset/._features.txt  
  inflating: /content/UCI HAR Dataset/features_info.txt  
  inflating: /content/__MACOSX/UCI HAR Dataset/._features_info.txt  
  inflating: /content/UCI HAR Dataset/README.txt  
  inflating: /content/__MACOSX/UCI HAR Dataset/._README.txt  
   creating: /content/UCI HAR Dataset/test/
   creating: /content/UCI HAR Dataset/test/Inertial Signals/
  inflating: /content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt  
   creating: /content/__MACOSX/UCI HAR Dataset/test/
   

In [21]:
features_df = pd.read_csv('/content/UCI HAR Dataset/features.txt',sep="\s+",
                         header=None, names=["column_index", "column_name"])

feature_names = features_df["column_name"].values.tolist()


In [37]:
path = "/content/UCI HAR Dataset/"

X_train = pd.read_csv(path+"train/X_train.txt",sep="\s+", header = None)
X_train.columns = feature_names
X_test = pd.read_csv(path+"test/X_test.txt",sep="\s+", header = None)
X_test.columns = feature_names
y_train = pd.read_csv(path+"train/y_train.txt",sep="\s+", header = None)
y_train.columns = ['action']
y_test = pd.read_csv(path+"test/y_test.txt",sep="\s+", header = None)
y_test.columns = ['action']



In [38]:
batch_size = 64
validation_split = 0.2

In [39]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)


In [40]:
y_train_array = np.array(y_train)
train_set = np.c_[X_train, y_train_array]

y_test_array = np.array(y_test)
test_set = np.c_[X_test, y_test_array]

In [41]:
X_train, y_train = split_sequences(train_set, 3)
print(X_train.shape, y_train.shape)

X_test, y_test = split_sequences(test_set, 3)
print(X_test.shape, y_test.shape)


(7350, 3, 561) (7350,)
(2945, 3, 561) (2945,)


In [42]:
time_period, sensors = X_train.shape[1], X_train.shape[2]
num_classes = 7

In [43]:
input_shape = time_period * sensors
X_train = X_train.reshape(X_train.shape[0], input_shape)
print("Input Shape: ", input_shape)
print("Input Data Shape: ", X_train.shape)
input_shape = time_period * sensors
X_test = X_test.reshape(X_test.shape[0], input_shape)
print("Input Shape: ", input_shape)
print("Input Data Shape: ", X_test.shape)

Input Shape:  1683
Input Data Shape:  (7350, 1683)
Input Shape:  1683
Input Data Shape:  (2945, 1683)


In [44]:

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [45]:
y_train_hot = to_categorical(y_train)
print("y_train shape: ", y_train_hot.shape)
y_test_hot = to_categorical(y_test)
print("y_test shape: ", y_test_hot.shape)

y_train shape:  (7350, 7)
y_test shape:  (2945, 7)


In [55]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(input_shape,1), activation='relu'))
model.add(Reshape((1, 1683, 32)))
model.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
model.add(Reshape((841, 64)))
model.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
model.add(Reshape((840, 192)))
model.add(GlobalAveragePooling1D())
model.add(BatchNormalization(epsilon=1e-06))
model.add(Dense(7))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 1683, 32)          4352      
                                                                 
 reshape_20 (Reshape)        (None, 1, 1683, 32)       0         
                                                                 
 conv1d_13 (Conv1D)          (None, 1, 841, 64)        4160      
                                                                 
 reshape_21 (Reshape)        (None, 841, 64)           0         
                                                                 
 conv1d_14 (Conv1D)          (None, 840, 192)          24768     
                                                                 
 reshape_22 (Reshape)        (None, 840, 192)          0         
                                                                 
 global_average_pooling1d_4  (None, 192)              

In [56]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train,
                    y_train_hot,
                    batch_size= 192,
                    epochs=10
                   )

Epoch 1/10
39/39 [==============================] - 78s 2s/step - loss: 1.1204 - accuracy: 0.4912
Epoch 2/10
39/39 [==============================] - 74s 2s/step - loss: 0.9272 - accuracy: 0.5988
Epoch 3/10
39/39 [==============================] - 74s 2s/step - loss: 0.8399 - accuracy: 0.6472
Epoch 4/10
39/39 [==============================] - 73s 2s/step - loss: 0.7498 - accuracy: 0.7004
Epoch 5/10
39/39 [==============================] - 74s 2s/step - loss: 0.6470 - accuracy: 0.7479
Epoch 6/10
39/39 [==============================] - 74s 2s/step - loss: 0.5758 - accuracy: 0.7924
Epoch 7/10
39/39 [==============================] - 73s 2s/step - loss: 0.5204 - accuracy: 0.8057
Epoch 8/10
39/39 [==============================] - 75s 2s/step - loss: 0.4650 - accuracy: 0.8298
Epoch 9/10
39/39 [==============================] - 75s 2s/step - loss: 0.4189 - accuracy: 0.8449
Epoch 10/10
39/39 [==============================] - 74s 2s/step - loss: 0.3837 - accuracy: 0.8627


In [57]:
# evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test_hot, batch_size=batch_size, verbose=0)
print("Accuracy", test_accuracy)

Accuracy 0.1799660474061966
